# Python Process Book for CS109 Project


## Overview and Motivation:

This project looks at trends in health topics over time.  The members of this project team are all public health students.  We were interested in how topics in health in a popular newspaper, The New York Times, have changed over time.  We decided to gain a deeper understanding of Latent Dirilecht Allocation (LDA) and use this topic modeling method to find the major topics in health over the past 5 decades (the period of time we were able to collect data for).  

Related work which inspired our project was Homework 5 for this class as well as Google Trends which show trends in Google searches over time for specific topics.

## Initial Questions:

 Which topics are persistent over time?  
 
 Which topics have a spike, when do they occur, and why did it happen? 
 Are there any surprising topics?
 
 With regards to LDA:
 Should we use Pattern (as in Hw5) or is there a better option for our data?
 What hyperparameters should we choose?
 
 Are there further analyses we can consider based on our findings?

## Data:

Data was pulled from the New York Times article API.  Using the API console (http://developer.nytimes.com/io-docs), we were able to inspect the type of results for a given query.  Originally, we decided to look at results using the 'fq=newsdesk:Health' option which would pull results under the Health topic section of the Times (approximately 680,000 documents since 1851).  However, after looking at the first 1000 results, we found that the documents pulled mainly consisted of videos, slideshows, and interactive features instead of articles.  We remedied this issue by instead using a query for the keyword 'Health' which searched all articles and their headlines for the word health.  Looking at the results from the API console showed that overall, using 'Health' as our query term instead of newsdesk:Health produced approiximately 40,000 more documents (720,000 total).

The Times API has several limiting factors when pulling data: 10,000 calls per day and a maximum of 100 pages per query.  To handle these limitations, our code pulled data by year and split each month into 3 parts (based on testing date ranges for number of pages which would be pulled so the total number of pages would be less than 100).  Year and count were entered manually to keep track of how many calls were being made and to break up the data calls for when errors occurred (such as timeouts, key errors, and date errors; when these errors were encountered the solution was appended to the code which resulted in the final version below).     

In this code, we requested the json dictionaries, used the relevant information to create a dataframe with the date, id, document type (article, blog, video), newsdesk section and subsection, and text from the title, abstract, and first paragraph.  The data was saved in a csv file for that section and tracked using an excel file - [DateTracker](https://github.com/dyan1211/teamsignificant/blob/master/DateTracker.xlsx).

At the beginning, we wanted to pull as much data as possible to get an idea of topic changes over time - in particular if there was a difference between today and some unspecified earlier time (articles range back to the mid-1800s).  Due to the limiting factors (and keeping in mind amount of time needed for analysis), we pulled data through the late 1950s.  We decided to perform our analysis on data from the end of October 2015 through 1966 the end of December 1966, giving us five decades of data.  

Text scraping and cleaning was done in https://github.com/dyan1211/teamsignificant/blob/master/DataScraping.ipynb

In [ ]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [ ]:
import json

In [ ]:
%%time
count = 1460 #enter new count starting number
year =  1954 #enter year
months = ['01', '01', '01', '02', '02', '02', '03', '03', '03', '04', '04', '04', '05', '05', '05', '06', '06', '06', '07', '07', '07', '08', '08', '08', '09', '09', '09', '10', '10', '10', '11', '11', '11', '12', '12', '12']
startdays = ['01', '11', '21', '01', '11', '21', '01', '11', '21', '01', '11', '21', '01', '11', '21', '01', '11', '21', '01', '11', '21', '01', '11', '21', '01', '11', '21', '01', '11', '21', '01', '11', '21', '01', '11', '21']
enddays = ['10', '20', '31', '10', '20', '27', '10', '20', '31', '10', '20', '30', '10', '20', '31', '10', '20', '30', '10', '20', '31', '10', '20', '31', '10', '20', '30', '10', '20', '31', '10', '20', '30', '10', '20', '31']

pcount = 0

for d in range(36):
    sdate = year + months[d] + startdays[d]
    edate = year + months[d] + enddays[d]

    docs=[]
    #get 1st page and number of documents
    url1 = "http://api.nytimes.com/svc/search/v2/articlesearch.json?q=Health&page=1&begin_date={}&end_date={}&api-key=5cdab36b05348a4da2e74046dfb16a03:17:73541790".format(sdate, edate) 
    lpage = requests.get(url1).json()['response']['meta']['hits']
    
    #calculate number of pages for call (divide total documents by 10 and add 2 in case not every page has 10 documents); 
    #if there are no hits skip to end
    if lpage is not 0:
        numpages = int(lpage/10 + 2) 
        pcount += numpages
    
        #get json files for first page
        pagedoc1 = requests.get(url1).json()['response']['docs']
        for j in range(0,len(pagedoc1)):
            docs.append(pagedoc1[j])    
    
        #get json dictionaries for rest of the pages
        for i in range(2, numpages): 
            url = "http://api.nytimes.com/svc/search/v2/articlesearch.json?q=Health&page={}&begin_date={}&end_date={}&api-key=5cdab36b05348a4da2e74046dfb16a03:17:73541790".format(i, sdate, edate)
            pagedocs = requests.get(url).json()['response']['docs']
            time.sleep(1)
        
            for j in range(0,len(pagedocs)):
                docs.append(pagedocs[j])

    #pull information from json file into dictionary        
        docsinfo = []
        for d in docs:
            obs = {}
            obs['id'] = d['_id']
            obs['type'] = d['type_of_material']
            obs['doctype'] = d['document_type']
            obs['date'] = d['pub_date']
            obs['news_desk'] = d['news_desk']
            obs['section'] = d['section_name']
            obs['subsection'] = d['subsection_name']
            obs['abstract'] = d['abstract']
            obs['paragraph'] = d['lead_paragraph']
        
            #Headline exceptions
            if d['headline'].get('main') is not None:
                obs['headline'] = d['headline']['main']
            elif d['headline'].get('name') is not None:
                obs['headline'] = d['headline']['name']
            else:
                obs['headline'] = ' '
    
            #get the date part of datetime
            if obs['date'] is not None:
                obs['date'] = obs['date'][0:10]
    
            #Remove empty abstract and lead paragraph cell to join text
            if obs['abstract'] is None:
                a = ' '
            else: 
                a = obs['abstract']
            if obs['paragraph'] == 'TK TK TK' or obs['paragraph'] is None:
                p = ' '
            else:
                p = obs['paragraph']
    
            #Join all the text columns
            text = [obs['headline'], p, a]
            obs['text'] = " ".join(text)
    
            docsinfo.append(obs)

    #create dataframe from dictionary, make date column date type, and store in csv
        docsdf = pd.DataFrame(docsinfo)
        docsdf['date'] = pd.to_datetime(docsdf['date'])
        docsdf.to_csv("data/docsdf-{}.csv".format(count), encoding = 'utf-8') 

    count += 1 
    

Next the files were concatenated into a single dataframe and saved to a csv file of our data which is located in our dropbox: https://www.dropbox.com/s/eedgwugamiw0zwd/total.csv?dl=0 

In [ ]:
#1966-2015
#There were no documents for 08/11/1978 - 10/31/1978 (leading to gap in csv files)
frames = []
for i in range(1,1338) : 
    dfs = pd.read_csv("docsdf-"+str(i)+".csv")
    frames.append(dfs)
for i in range(1346,1784) : 
    dfs = pd.read_csv("docsdf-"+str(i)+".csv")
    frames.append(dfs)
totaldf = pd.concat(frames)
totaldf['date'] = pd.to_datetime(totaldf['date'])

In [ ]:
totaldf.to_csv("total.csv", index=False)

We looked at the types of documents to see if there were any trends and found that the majority of the documents were articles.  We are mainly interested in articles, and two of the document types are more recent types (multimedia and blogpost), so we decided to only include articles in our corpus.

Document analysis was done in https://github.com/dyan1211/teamsignificant/blob/master/Exploration.ipynb

In [ ]:
# Code to look at frequencies of document types
sns.set(style="white", context="talk")
ax = sns.barplot(x=('Article', 'Blogpost', 'Column', 'Multimedia', 'Recipe'), y=type_counts)
ax.set(title="Document Type Frequencies",ylim=(0,400000),yticks=[100000,200000,300000,400000])
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x()+0.2, height+10000, '%d'%height, fontsize=14)
#ax.text(-0.2,380000, "369,890", fontsize=14)
#ax.text(0.82,40000, "31,369", fontsize=14)
sns.despine(bottom=True)

![](http://i.imgur.com/qLMXD3e.png)

In [ ]:
#Only use documents that were specified as article will be used
df = totaldf[totaldf['doctype'] == 'article']

After plotting our data over time and checking days which had highest number of articles, we found that the years 1980 and 2014 had some strange spikes.  We realized there were a number of duplicate documents in our data which had pieces of another documents (such as a shorted title or only the first few sentences of a paragraph). We utilized the drop_duplicates function to by creating a new column which contained the first 50 characters of the paragraph column in order to capture the matching sections of the two documents.

Duplicate dropping was done in https://github.com/dyan1211/teamsignificant/blob/master/DuplicateArticles.ipynb

In [ ]:
#Bar graph of number of documents per year before dropping duplicates (spike in 1980 and 2014)
years = df.year.value_counts().sort_index()
years.plot(kind='bar')

![](http://imgur.com/rVyNJx1.png)

In [ ]:
#Check dates with most articles 
days = articledf.date.value_counts()
days[:20].plot(kind='barh');

![](http://imgur.com/caAkxtg.png)

In [ ]:
#Remove duplicates
df['dupcheck'] = df['paragraph'].str[0:50]
df = df.drop_duplicates('dupcheck')

In [ ]:
#Bar graph of number of documents per year after dropping duplicates (smoothed out 1980, though still 2014 spike )
years = df.year.value_counts().sort_index()
years.plot(kind='bar')

![](http://imgur.com/dHtCfzN.png)

This graph looks much better though there is still the 2014 spike.  Using the API console through NYT, I check the total number of articles in 2014 compared to 2013 (not just 'Health' articles):

Total # of articles produced by NYT in 2013 = 117593

Total # of articles produced by NYT in 2014 = 308867

So the spike is not specific to our search.  We do not have a concrete explanation, though we considered the possibility that it is due to an increase in web content.

#### Choice of Time Periods

Before moving forward, we made the decision that in order to parse out clear and specific topics we should perform LDA on subsets of the data.  By using the entire dataset, we were concerned that the topics would end up being too general.  We decided to use 5 year periods - large enough to have data to run LDA on and to capture topics which would have a trend in time, but small enough that some major events would show up. 

The code for splitting the df was done in https://github.com/dyan1211/teamsignificant/blob/master/Textblob.ipynb

## Exploratory Data Analysis:

We used spark to clean and analyze our data since these processes are easily parallelized.  Spark was implemented using homebrew on a Mac. 

In [ ]:
import os
os.environ['PYSPARK_PYTHON'] = '/Applications/anaconda/bin/python'

In [ ]:
import findspark
findspark.init()
print findspark.find()

In [ ]:
import pyspark
conf = (pyspark.SparkConf()
    .setMaster('local')
    .setAppName('pyspark')
    .set("spark.executor.memory", "2g"))
sc = pyspark.SparkContext(conf=conf)

In [ ]:
import sys
rdd = sc.parallelize(xrange(10),10)

In [ ]:
from pyspark.sql import SQLContext
sqlsc=SQLContext(sc)

We began by cleaning our code in order to perform LDA.  We decided to use NLTK to process our text since it is the leading platform for natural language processing.  However, we found that NLTK takes an extremely long time to process our large data set.  Instead, we utilized TextBlob which works off of both NLTK and Pattern platforms.  TextBlob also has the advantage of implementing the Averaged Perceptron algorithm which has been shown to be faster and more accurate than NLTK and Pattern (http://stevenloria.com/tutorial-state-of-the-art-part-of-speech-tagging-in-textblob/).

The text was tokenized, tagged for part of speech, made all lowercase, lemmatized, and the nouns were extracted.  

We considered issues with making words all lowercase due to some nouns such as AIDs and WHO but decided that not using .lower would potentially cause more problems.  In addition, the AP tagger should be able to identify nouns based on the sentence structure and even if a word has multiple meanings, if that word is mostly used in a specific way (HIV/AIDs vs government aid), it should not be difficult to identify based on the other words which make up the topic and the documents associated with it.  (In fact, we were able to distinguish between the two in our analysis).

The initial NLTK work was done in >>>>>>

TextBlob work (and the following parseout work) was done in https://github.com/dyan1211/teamsignificant/blob/master/Textblob.ipynb

In [ ]:
from textblob import TextBlob as tb
from textblob_aptagger import PerceptronTagger
from textblob import Blobber

In [ ]:
import nltk
from nltk.corpus import stopwords #stopwords

In [ ]:
#The necessary NLTK packages need to be downloaded to implement some of the functions. RUN ONLY ONCE
nltk.download()

In [ ]:
#Set up Averaged Perceptron Tagger for TextBlob
tb = Blobber(pos_tagger=PerceptronTagger())

In [ ]:
#Clean the text for each document to extract the nouns 
#Cleaning includes lemmatization and removal of stopwords

def get_parts(thetext):
    nouns=[]
    tagged = tb(thetext).tags # a list of tuples
    for tup in tagged:
        w, tag = tup  
        if tag in ['NN', 'NNS', 'NNP', 'NNPS']:
            word = w.lemmatize().lower()
            if word[-1] in punctuation : 
                word = word[:-1]
            if word in stops or word in punctuation or len(word)==1 :
                continue
            nouns.append(word)
    nouns2=[]
    for n in nouns:
        if len(n)!=0:
            nouns2.append(n)
    return nouns2

Parse out the nouns and organize in a list for each document.  We decided to parse on the document level (instead of the sentence level) under the assumption that each document could be classified as a topic (or a few topics).

*This code was run separately for each 5 year period - all example output is for 1966

In [ ]:
%%time

parseout = []
for index, row in df.iterrows() : 
    parseout.append(get_parts(row.text))

In [ ]:
# saving parseout to txt file
with open("parseout.txt", 'w') as f:
    json.dump(parseout,f)

Using the output for each 5-year time period, we were able to create word clouds to get an idea of the words in our corpus.  Not surprisingly, we can see that generalized words dominate - new, city, state, today.  But we can also pick out the presidents during most periods and some health related words also jump out - program, hospital, doctor. 

Wordcloud work was done in https://github.com/dyan1211/teamsignificant/blob/master/Textblob.ipynb

Code is based on https://github.com/amueller/word_cloud

In [ ]:
from os import path
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

from wordcloud import WordCloud, STOPWORDS

In [ ]:
# read the mask image taken from http://static01.nyt.com/images/icons/t_logo_291_black.png
nyt_mask = np.array(Image.open("~/nyt_mask_2.png"))

In [ ]:
wc = WordCloud(background_color="white", max_words=2000, mask=nyt_mask, stopwords=STOPWORDS)

In [ ]:
# create single string of all text for each parseout list
import itertools
document1966 = list(itertools.chain.from_iterable(parseout1966))
document1966 = ' '.join(document1966)
wc.generate(document1966)  #generate wordcloud

# show
plt.imshow(wc)
plt.axis("off")
plt.figure()
plt.show()

![](http://i.imgur.com/m7nfPPr.png)

### LDA 

LDA was done in >>>>>>

We started by created a corpus of our words.

In [ ]:
from gensim import corpora, models, similarities, matutils

In [ ]:
documents = parseout

In [ ]:
dictionary = corpora.Dictionary(documents)
dictionary.filter_extremes(no_below=5,no_above=0.75,keep_n=100000)
dictionary.compactify()

In [ ]:
corpus = [dictionary.doc2bow(document) for document in documents]

We wanted a way to decide how many topics would be best for our data.  In addition, since each of our 5-year time periods had a different number of documents, we assumed they were likely to also have a different number of topics (or the corpus for that period may be able to support more topics based on its content). Ideally, we would check the topic output after using several different options for number of topics and decide which worked best for our data.  Due to time constraints, we searched for an automatic method. After searching for different ways to approach this task, we found a method to find the number of topics using  KL divergence based on the idea that we can view LDA as a matrix factorization.  

Code for this section (and details for the method) are based on: http://blog.cigrainger.com/2014/07/lda-number.html

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
def sym_kl(p,q):
    return np.sum([stats.entropy(p,q),stats.entropy(q,p)])

In [ ]:
def arun(corpus,dictionary,min_topics,max_topics,step):
    kl = []
    for i in range(min_topics,max_topics,step):
        lda = models.ldamodel.LdaModel(corpus=corpus,
            id2word=dictionary,chunksize=170,num_topics=i)  #we decided on a chunksize of approximately .01 of number of documents
        m1 = lda.expElogbeta
        U,cm1,V = np.linalg.svd(m1)
        #Document-topic matrix
        lda_topics = lda[corpus]
        m2 = matutils.corpus2dense(lda_topics, lda.num_topics).transpose()
        cm2 = l.dot(m2)
        cm2 = cm2 + 0.0001
        cm2norm = np.linalg.norm(l)
        cm2 = cm2/cm2norm
        kl.append(sym_kl(cm1,cm2))
    return kl

In [ ]:
l = np.array([sum(cnt for _, cnt in doc) for doc in corpus])
kl = arun(corpus,dictionary,min_topics=1,max_topics=100,step=5)  #searched best number of topics between 1 and 100 by 5

In [ ]:
# Plot kl divergence against number of topics
x = np.arange(1,100,5)
plt.plot(x, kl)
plt.ylabel('Symmetric KL Divergence')
plt.xlabel('Number of Topics')
plt.savefig('kldiv1966.png', bbox_inches='tight')

![](http://i.imgur.com/9aRvg9m.png)

For the LDA we also needed to decide on the number for chunksize (the number of documents which are loaded into memory at a time).  Using 1966 as an example (with number of topics equal to 50), we tested chunksizes of 10%, 5%, and 1% of the total document number.  After inspecting the topic output, we found that smaller chunksizes (1% of document total) provided clearer topics (the grouped words told a clearer story on average for this chunksize).  

Therefore, for each period, we ran LDA using the number of topics chosen from above and a chunksize of 1% of document size.  

The output from our LDA (lda.print_topics) can be found [here](https://docs.google.com/a/mail.harvard.edu/document/d/12jsDy1T5_7QjslxpsDnzfzH60H00rV0YEPizmwzknNU/edit?usp=sharing)

We had a total of 520 topics - 50 topics on average for each period.

In [ ]:
import gensim

In [ ]:
lda1966 = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary,chunksize=170, num_topics=50)

In [ ]:
lda1966.print_topics(num_topics=50)

`[u'0.146*law + 0.102*may + 0.066*jersey + 0.044*new + 0.035*car + 0.034*state + 0.032*today + 0.025*critic + 0.023*code + 0.022*nj',
 u'0.138*agency + 0.135*department + 0.087*dept + 0.041*federal + 0.036*employes + 0.036*government + 0.034*u.s + 0.032*standard + 0.025*farm + 0.013*asthma',
 u'0.055*chief + 0.053*pay + 0.050*contract + 0.045*pact + 0.041*cost + 0.041*wage + 0.033*charles + 0.029*girl + 0.026*river + 0.025*ranch',
 u'0.112*south + 0.093*education + 0.082*plan + 0.042*feb + 0.040*line + 0.033*appeals + 0.033*southern + 0.033*opposition + 0.031*action + 0.028*ap',
 u'0.256*state + 0.059*nys + 0.050*gov + 0.046*unit + 0.038*albany + 0.020*race + 0.019*interest + 0.016*term + 0.016*connecticut + 0.016*governors',
 u'0.067*year + 0.060*disease + 0.053*heart + 0.051*dec + 0.048*work + 0.038*dies + 0.037*attack + 0.034*dr + 0.026*expert + 0.025*funds',
 u'0.215*johnson + 0.171*president + 0.078*oct + 0.049*washington + 0.029*change + 0.026*policy + 0.026*april + 0.022*capital + 0.018*set + 0.018*rule',
 u'0.223*city + 0.099*council + 0.062*budget + 0.060*rate + 0.028*year + 0.028*part + 0.028*francis + 0.026*increase + 0.026*rise + 0.026*yesterday',
 u'0.082*office + 0.057*poverty + 0.039*o'connor + 0.037*dec + 0.034*foundation + 0.033*gen + 0.031*suit + 0.028*poor + 0.028*abortion + 0.027*fed',
 u'0.069*hospitals + 0.063*police + 0.050*grant + 0.045*cut + 0.043*hotel + 0.035*gardner + 0.034*commission + 0.031*pub + 0.030*urban + 0.030*pressure',
 u'0.078*public + 0.065*act + 0.046*today + 0.041*upi + 0.034*calif + 0.031*team + 0.029*phs + 0.026*visit + 0.025*service + 0.023*inquiry',
 u'0.041*way + 0.027*year + 0.026*roosevelt + 0.022*finds + 0.022*activity + 0.020*prime + 0.020*renewal + 0.020*peking + 0.020*thursday + 0.018*fla',
 u'0.096*war + 0.038*march + 0.037*peace + 0.036*today + 0.032*secretary + 0.031*defense + 0.026*us + 0.026*world + 0.023*foreign + 0.021*threat', 
 u'0.077*birth + 0.072*control + 0.055*director + 0.049*woman + 0.046*use + 0.029*children + 0.026*back + 0.026*service + 0.023*data + 0.021*birthday',
 u'0.051*authority + 0.048*columbia + 0.045*season + 0.035*sale + 0.034*bank + 0.032*source + 0.028*pope + 0.027*stress + 0.024*lag + 0.021*named',
 u'0.093*party + 0.073*leader + 0.057*campaign + 0.054*senator + 0.038*candidate + 0.035*election + 0.031*l.i + 0.022*nassau + 0.020*leaders + 0.020*g.o.p',
 u'0.058*job + 0.056*water + 0.050*survey + 0.047*area + 0.043*men + 0.041*space + 0.035*brooklyn + 0.032*chairman + 0.019*league + 0.019*vaccine',
 u'0.078*welfare + 0.061*conf + 0.058*news + 0.036*com + 0.036*conference + 0.030*today + 0.030*committee + 0.029*white + 0.027*washington + 0.026*proposal',
 u'0.148*school + 0.050*court + 0.042*student + 0.042*negro + 0.037*order + 0.028*ct + 0.027*rights + 0.026*today + 0.025*federal + 0.025*fed',
 u'0.127*study + 0.066*report + 0.064*research + 0.042*operation + 0.037*natl + 0.036*problem + 0.035*group + 0.034*condition + 0.032*panel + 0.024*inst',
 u'0.054*john + 0.050*bellevue + 0.049*post + 0.049*china + 0.041*mao + 0.031*queens + 0.026*communist + 0.023*park + 0.021*summer + 0.020*story',
 u'0.026*sept + 0.022*july + 0.015*college + 0.015*nurse + 0.014*mr + 0.012*tex + 0.012*help + 0.010*today + 0.010*saigon + 0.009*equipment',
 u'0.045*security + 0.038*social + 0.036*americans + 0.035*soc + 0.032*econ + 0.032*business + 0.031*benefit + 0.028*article + 0.027*need + 0.025*growth',
 u'0.342*program + 0.070*administration + 0.050*adm + 0.030*federal + 0.029*u.s + 0.022*future + 0.022*javits + 0.020*washington + 0.015*priority + 0.012*harry',
 u'0.113*medicare + 0.094*home + 0.052*care + 0.046*death + 0.043*person + 0.038*question + 0.036*mass + 0.036*surgery + 0.030*fire + 0.029*message',
 u'0.147*nov + 0.120*dec + 0.043*rise + 0.042*today + 0.035*price + 0.031*cent + 0.029*ap + 0.027*increase + 0.026*san + 0.025*washington',
 u'0.093*problem + 0.086*system + 0.083*nation + 0.076*pres + 0.050*support + 0.044*crisis + 0.041*us + 0.040*int + 0.035*shortage + 0.028*london',
 u'0.131*center + 0.085*patient + 0.053*medicaid + 0.048*community + 0.046*drs + 0.039*nyc + 0.038*money + 0.031*clinic + 0.029*physician + 0.027*medicine',
 u'0.049*comr + 0.049*st + 0.046*building + 0.038*commissioner + 0.035*age + 0.033*population + 0.031*room + 0.030*field + 0.027*youth + 0.024*yesterday',
 u'0.114*time + 0.048*end + 0.045*airline + 0.039*legislation + 0.033*factor + 0.030*merger + 0.025*key + 0.025*animal + 0.024*agreement + 0.021*md',
 u'0.217*hospital + 0.084*hosp + 0.050*dr + 0.037*university + 0.033*yesterday + 0.028*east + 0.027*harlem + 0.022*series + 0.021*charge + 0.020*manhattan',
 u'0.100*union + 0.089*strike + 0.076*labor + 0.047*hearing + 0.044*worker + 0.042*dispute + 0.040*nurses + 0.031*progress + 0.030*demand + 0.027*amendment',
 u'0.065*company + 0.054*safety + 0.041*statement + 0.041*co + 0.040*robert + 0.036*international + 0.031*force + 0.029*industry + 0.025*resignation + 0.024*map',
 u'0.100*dr + 0.093*drug + 0.061*food + 0.056*effect + 0.042*doctor + 0.038*society + 0.028*research + 0.027*washington + 0.024*scientist + 0.021*lsd',
 u'0.190*city + 0.101*lindsay + 0.087*mayor + 0.053*yesterday + 0.033*service + 0.027*emergency + 0.021*resident + 0.020*bronx + 0.019*plant + 0.019*plan',
 u'0.125*vietnam + 0.062*housing + 0.053*test + 0.036*facility + 0.033*rev + 0.031*level + 0.030*training + 0.028*rusk + 0.025*tomorrow + 0.022*number',
 u'0.124*life + 0.056*result + 0.053*mental + 0.044*n.j + 0.026*accident + 0.026*dr + 0.024*discussion + 0.024*figure + 0.019*yale + 0.019*retirement',
 u'0.166*aid + 0.140*bill + 0.090*hosps + 0.050*right + 0.040*vote + 0.035*treatment + 0.025*today + 0.024*legis + 0.022*urge + 0.021*com',
 u'0.163*fund + 0.083*world + 0.074*project + 0.071*national + 0.069*head + 0.052*chicago + 0.030*construction + 0.030*william + 0.029*republican + 0.019*measure',
 u'0.149*united + 0.120*states + 0.044*son + 0.044*jr + 0.042*daughter + 0.033*army + 0.027*history + 0.025*shriver + 0.025*america + 0.022*james',
 u'0.085*drive + 0.061*record + 0.040*george + 0.033*hope + 0.026*theater + 0.026*high + 0.026*decade + 0.025*town + 0.022*reports + 0.022*sea',
 u'0.122*pollution + 0.115*air + 0.063*control + 0.058*role + 0.041*fight + 0.027*step + 0.026*exec + 0.025*first + 0.023*issue + 0.020*north',
 u'0.110*day + 0.067*man + 0.066*family + 0.055*illus + 0.049*planning + 0.047*night + 0.029*por + 0.020*year + 0.018*beach + 0.016*goal',
 u'0.138*rockefeller + 0.104*governor + 0.036*insurance + 0.029*yesterday + 0.028*tour + 0.027*costello + 0.026*catholic + 0.026*state + 0.025*session + 0.023*plan',
 u'0.127*mrs + 0.086*county + 0.082*child + 0.056*miss + 0.035*aug + 0.027*staff + 0.022*teacher + 0.019*mother + 0.018*parent + 0.018*philadelphia',
 u'0.082*june + 0.064*cabinet + 0.061*kennedy + 0.057*tax + 0.043*government + 0.031*today + 0.030*govt + 0.026*min + 0.024*gift + 0.021*little',
 u'0.198*new + 0.106*york + 0.043*medical + 0.035*city + 0.034*case + 0.031*american + 0.030*yesterday + 0.028*assn + 0.027*association + 0.024*ny',
 u'0.092*senate + 0.081*thaler + 0.078*sen + 0.069*brown + 0.050*angeles + 0.049*los + 0.038*move + 0.032*finance + 0.030*plans + 0.027*coast',
 u'0.085*board + 0.061*bd + 0.058*site + 0.056*meeting + 0.051*power + 0.048*talk + 0.037*general + 0.032*island + 0.031*executive + 0.028*california',
 u'0.149*house + 0.078*washington + 0.075*congress + 0.058*cancer + 0.041*today + 0.039*hr + 0.037*cigarette + 0.035*cong + 0.027*advertising + 0.023*ad']`

#### Visualization of topics

pyLDAvis is a python libarary used previously to visualize topic models. The package extracts information from a fitted LDA topic model to inform an interactive web-based visualization. The package information can be found here https://pypi.python.org/pypi/pyLDAvis

In [ ]:
import pyLDAvis.gensim as gensimvis
import pyLDAvis

In [ ]:
vis_1966 = gensimvis.prepare(lda1966, corpus, dictionary)
pyLDAvis.display(vis_1966)

![](http://i.imgur.com/dXcRcWj.png)

#### Creating new data frame 

The idea is to get the count of articles of a topic within each time period and then plot it over time to understand the trend of the popularity of a topic. Instead of assigning an article one single topic, we consider it contributing to several topics based on the percent that it belongs to a certain topic, which we considered a better way as a weigted sum to measure the popularity of topics. In order to do this, we created a new data frame whose columns are `Topics` and rows are `Articles`, with cells as the corresponding percent of each article under each topic, and then appended the data frame to each 5-year data frame (to match up the `Date`). 

In [ ]:
n = 50
topicdf = pd.DataFrame()
topicdf = np.zeros(shape=(len(corpus),n))
for index, bow in enumerate(corpus) : 
    topics = lda1966.get_document_topics(bow)
    for t in topics :
        topicdf[index][t[0]] = t[1]

In [ ]:
topicdf1966 = pd.DataFrame(topicdf,columns=["Topic"+str(i) for i in range(0,n)])
comb1966 = pd.concat([df1966, topicdf1966], axis=1)

Checking topics - we looked at the top 10-20 articles with the highest percent for a topic to see the specific articles which fell under the topic (useful for unclear topics and identifying specific events).

In [ ]:
import heapq

In [ ]:
#Get the top 20 articles with the highest percent for a topic to 
top20 = [list(fulldf['Topic1']).index(p) for p in heapq.nlargest(20, list(fulldf['Topic1']))]
list(fulldf['text'][top20])[0:19]

## Final Analysis:

#### Sum up the probabilities and standardize by total # of articles

Since we have different number of articles within each time period as we can see from the graph in the beginning, to make them comparible, we summed up the probabilities of each topic through out all articles and standardized it by the total number of articles within that certain time range. We tried to group the data by month at first, but the plots turned out to have too much noise. Therefore, we changed our time grid into quarter to group the data by quarter and created time series data based on the date and mean probabilities within that quarter.

In [ ]:
ts = comb1981.set_index('date').resample('Q')['Topic10']

#### Plotting each topic

Besides picking up interesting topics based on the Probability and Text from LDA output, we also considered looking at the Trend of topic without knowing any of the text, which actually gave us very instersting results since it lowers the bias of subjective sense based on the texts themselves. We plotted every single topic over time to see how the popularities change over time. We considered those who have an apparent regular pattern (periodic/increasing/decreasing) and those who have an unusual spike at some point as potential interesting topics or events, and then take a further look at the documents as well as historical evidence for them.

In [ ]:
def plottopic(df, n) : 
    fig, axes = plt.subplots(nrows=n/5, ncols=5, figsize=(15, int(n)/2), tight_layout=True)
    for i in range(n) : 
        ts = df.set_index('date').resample('Q')['Topic'+str(i)]
        ax = axes[int(i)/5][i-5*(int(i)/5)]
        ax.plot(ts,label="Topic%s" % i)
        ax.legend(frameon=False, loc="upper right")

In [ ]:
plottopic(comb1966,50)

![](http://i.imgur.com/bsKQKgk.png)

#### Compared with Google Trends

Google Trends only have data from 2005, so we picked three intereting findings in health related topics from recent years, which are Swine Flue, Ebola and ACA(Health Bill), to compare the LDA topic modeling based on New York Times articles and the popularity of terms in Google trends. The numbers that appear in Google Trends data show total searches for a term relative to the total number of searches done on Google over time (scaled from 0 to 100). It is technically a relative popularity of the term. We managed the score from Google trends to a comparible scale with the probabilities we had from LDA, and plotted them together. The result shows that they conincide with each other very well. 

The code below is an example of Swine Flu. The entire process can be found at >>>>>>>>>>>>>

In [ ]:
event_swine = comb2006.set_index('date').resample('Q')['Topic34']
swine = pd.read_csv("swine.csv")
swine['Week'] = pd.to_datetime(swine['Week'])
swines = swine.set_index('Week').resample('Q')['swine flu']/600
swinedf = pd.DataFrame({"NYT":event_swine,"Google Trends":swines},index=event_swine.index)

In [ ]:
with plt.style.context('fivethirtyeight'):
    plot = swinedf.plot()
    plot.set_title("Swine Flu - LDA Topic Modeling vs Google Trends Popularity", fontdict=font3)
    plot.tick_params(axis='both', which='major', labelsize=15) # increase xlabel fontsize
    plot.axes.get_yaxis().set_ticks([]) # remove yticks
    plot.set_xlabel("") # remove xlabel
    plot.grid(False) # remove grid
    plt.savefig('swineflu.png', bbox_inches='tight')

#### Swine Flu

![](http://i.imgur.com/CSoF5MS.png)

#### Patient Protection and Affordable Care Act

Unlike Swine Flu and Ebola, the plot of Health Bill shows a little bit lagged reaction of general attention than New York Times, which actually makes sense because the other two are both disease outbreak that spread very fast while Health Bill is a politics related topic which takes time for people to really pay attention on. 

![](http://i.imgur.com/a2mgS0C.png)

#### Ebola

![](http://i.imgur.com/K97Td1J.png)

## Presentation: